In Lecture 9, we have learned about multi-agent systems and adverserial games. A central algorithm in game playing is **MiniMax**.

MiniMax essentially searches the entire space of moves possible by both players at any given time till the game reaches an end. This is how it maximizes reward while the opponent is trying to minimize it. We saw an optimization called alpha-beta pruning for MiniMax which is a great improvement but only by a factor of 2. This might help in games such as chess (an AI agent can be twice as smart on the same computer now assuming it uses a plausible heuristic).

TicTacToe has $9! = 362880$ possible states, roughly half of them are valid states. This kind of state space can be fully searched by computers.

Some games such as Go are way too complicated.

By [one estimate](https://www.google.com/search?q=2081681993819799846994786333448627702865224+5388453054842563945682092741961273801537852+5648451698519643907259916015628128546089888+314427129715319317557736620397247064840935&filter=0&biw=1280&bih=627) there are:

```
2081681993819799846994786333448627702865224
5388453054842563945682092741961273801537852
5648451698519643907259916015628128546089888
314427129715319317557736620397247064840935
```

States!

That certainly cannot be handled by even the biggest supercomputers or an earth filled with GPUs.

A big search tree!

## Infeasible state space

Go is a [googol](https://en.wikipedia.org/wiki/Googol) ($10^{100}$) times more complex than chess. It has more states than there are atoms in the universe. It has been a formidable challenge for AI.


In March 2016 DeepMind's AlphaGo Lee became the first program to beat a human expert -- 18-time world champion Lee Sedol. (DeepBlue beat Gary Kasparov in chess in 1997 -- super human ability in Go took 19 years longer).

The key insight of this software was to use a neural network to learn the game by looking at how humans play (using plays from ameteur players on internet Go servers) it to begin with, but later by playing the program against itself.

In 19 Oct 2017 DeepMind unweiled a newer, more elegant version of AlphaGo -- the AlphaGo Zero. This version learnt the game all on its own through playing millions of games with its best self. In 3 days, AlphaGo Zero trained enough to beat the first version (AlphaGo Lee) 100 games to 0!

## AlphaGo Zero's algorithm

### Background

**Q-learning** with neural networks:

- **Goal-orientation:** The agent's goal is to figure out what is the best move at any given state.
- Recall the $Q$ function we used for value iteration (it's the expected future reward). $Q : S \times A \rightarrow \mathbb{R}$
- Usually we represent $Q$ as a table: we actually store a value for every action at every action. But as you can tell in the case of Go, $Q$ is going to be too big!
- **One idea is to approximate $Q$ with a function that requires less storage.** A function such as a neural network.
  - Start off with a random neural network
  - Keep playing the game to improve the $Q$ function it approximates
  
This is not exactly what AlphaGo zero uses, but it can be used to play a lot of games including Breakout. AlphaGo Zero marries Q-learning with Monte-carlo tree search.

**Monte carlo tree search (MCTS)**


![](https://nikcheerla.github.io/deeplearningschool//media/branching.jpg)

One way programs can search the state space graph is to **pick a random sample of branches**, playing each one many times to see which ones have a better probability of winning down the tree. This is called Monte-carlo tree search.

**Approximating a function that guides MCTS**

AlphaGo zero creates a neural network $f_{\theta}$ which approximates the probability distribution among all branches for the next move. Then the sampling algorithm draws from this probability distribution. It also uses the same neural network to guess the expected probability of winning from the state it is applied to.


$$(\textbf{p}, v) = f_{\theta}(s)$$

Here $\textbf{p}$ maps every action available at state $s$ to a probability value that it should be sampled.



**Self-play**

A neural network needs to be told what are good moves and what are not in order to train itself to be better at the game. AlphaGo Zero keeps improving by playing against itself. Winning the game gets a reward of 1, losing
This is the simple algorithm that runs all of AlphaGo Zero!

## Notes about implementation

For the purpose of this presentation we chose [AlphaGo.jl](https://github.com/tejank10/AlphaGo.jl) a Julia implimentation of AlphaGo Zero algorithm. The package not only implements the algorithm for Go, but also for the game of [Gomoku](https://en.wikipedia.org/wiki/Gomoku) which is a **generalization of Tic-tac-toe** played on the Go board. The program allows you to tweak the size of the board and the number of squares that need to be taken up consequitively to win the game. Hence Tic Tac Toe is a special case of Gomoku when the board is of size 3x3 and you need 3 colinear squares to win.

We tweaked the training to create a smaller neural network (order 48 instead of 256), since our problem is considerably smaller and ran the training several times with different parameters.

Parameters we tried:

- 48x48 convolutional layers that are 6 levels deep, trained on 10000 games, but we ended up sampling 6200 initial games that we used as prior for our mote-carlo sampling. Most games were run without training, but the neural network quickly learned the optimal strategy once training began
- 48x48 convolutional layers- 6 levels deep, trained on 500 games with 81 games as the starting sample for Monte-carlo priors. (This is the version you can play below -- it has noticable variablility in the smartness from beginning to end)

##### Setting up Julia packages

The following line should install all the dependencies to run this notebook. Ignore any build errors to do with CUDAnative and CUDAdrv packages, these are not required but are installed by default by the AlphaGo implementation.

You will have to restart the Jupyter notebook server and start this notebook again to get some of the UI packages to work (this will let you play Tic tac toe against agents of different skill levels :-) )

In [1]:
add WebIO JSExpr InteractBase CSSUtil Observables https://github.com/shashi/AlphaGo.jl.git#4cdb733 Flux BSON

LoadError: syntax: extra token "WebIO" after end of expression

In [2]:
# A function to load saved models

using AlphaGo, Flux
using BSON: @load

function load_model(str, n, env::AlphaGo.GameEnv)
  @load str*"/agz_base-$n.bson" bn
  @load str*"/agz_value-$n.bson" value
  @load str*"/agz_policy-$n.bson" policy

  @load str*"/weights/agz_base-$n.bson" bn_weights
  @load str*"/weights/agz_value-$n.bson" val_weights
  @load str*"/weights/agz_policy-$n.bson" pol_weights

  Flux.loadparams!(bn, bn_weights)
  Flux.loadparams!(value, val_weights)
  Flux.loadparams!(policy, pol_weights)

  NeuralNet(env; base_net=bn, value=value, policy=policy)
end

load_model (generic function with 1 method)

In [3]:
include("ui.jl") # The game-playing UI code

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

play_with (generic function with 1 method)

### Playing against the neural network

Below is a UI where you can play against the neural network at various stages of its learning.

In the cell immediately below, the neural network has been only trained on 10 games.

The agent is smart to win or draw most games already, but try playing (1,1), (3,3), (1, 3), (1, 2) -- you will win this game. (note: $(i,j)$ is $i^{th}$ row $j^{th}$ column.)

In the subsequent cells you can adjust the level of training the Neural network has had and play the same game to see where the Neural network figured out to draw this game.

In [24]:

# play with the dumbest network first:
tictactoe = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe

# there are 50 levels we have trained, second argument is the level
nn = load_model("models_48_81boot", "10", tictactoe)


# play_with lets you play the game with any given neural network
b,t = play_with(tictactoe, nn)
b # -- this is the board, and gets interactively displayed

# NOTE: once you click on a square it may take several seconds for the first move to be registered
# Julia is a just-in-time compiled language, the first move compiles the whole
# program to C-like machine code, hence be patient

Scope("scope-a5a4f3b7-b15b-4445-b4c2-0c8697a5169f", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-89de6437-cf26-4cf6-ad10-de898a843329", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_623", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"clicks\",\"scope\":\"scope-a5a4f3b7-b15b-4445-b4c2-0c8697a5169f\",\"id\":\"ob_621\",\"type\":\"observable\"},[1,1])})")),:style=>Dict("height"=>36.0px,"borderRadius"=>4.0px,"width"=>36.0px,"margin"=>2.0px)), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}("ob_632", 0, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "clicks")))]), nothing)), Set(String[]), nothing, Any["knockout"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-89de6437-cf26-4cf6-ad10-de898a843329\",\"id\":\"ob_632\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)))], Dict{Symbol,Any}(:className=>"field"), 1), Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-0d0cfbfd-c741-4b8f-90a5-42f87780cf67", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_626", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"cli

### Play with an agent which has various levels of training!

In [29]:
@manipulate for level = slider(1:50, label="Experience level (10 games each): ", value=1)
    gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
    nn = load_model("models_48_81boot", string(level), gametype) # there are 50 levels we have trained, second argument is the level

    play_with(gametype, nn)[1]
end

Widget{:manipulate,Any}(DataStructures.OrderedDict{Symbol,Any}(:level=>Widget{:slider,Int64}(DataStructures.OrderedDict{Symbol,Any}(:changes=>Observable{Int64}("ob_797", 0, Any[SyncCallback(Scope("knockout-component-a06bb5d6-cd1e-4154-8fb7-92d7b847c3fa", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["Experience level (10 games each): "], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>50,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64}(#= circular reference @-6 =#), nothing),"value"=>(Observable{Int64}("ob_796", 1, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "value"))), #g#15{##16#17{##34#36,Observable{Any}},Tuple{Observable{Int64}}}(##16#17{##34#36,Observable{Any}}(##34#36(), Observable{Any}("ob_820", Scope("scope-994eb8b8-2cee-4ab2-a94b-0c88f79f9159", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-81cc1c9d-e51e-4ea4-a237-ff5a2014a03f", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_800", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"clicks\",\"scope\":\"scope-994eb8b8-2cee-4ab2-a94b-0c88f79f9159\",\"id\":\"ob_798\",\"type\":\"observable\"},[1,1])})")),:style=>Dict("height"=>36.0px,"borderRadius"=>4.0px,"width"=>36.0px,"margin"=>2.0px)), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}("ob_809", 0, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "clicks")))]), nothing)), Set(String[]), nothing, Any["knockout"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)

In [30]:
include("optimal.jl")

play_optimal (generic function with 1 method)

## Comparison against optimal player over time

The true test of skill for the neural network is comparison against the optimal policy for the game. Fortunately, in the game of Tic tac toe we can actually have do this. We have written an optimal player in `optimal.jl` file, it is in the same vein as the one in Homework 6. If you are curious about how the Julia code looks, you may have a look in there. We also have a function play_optimal which will play against a given neural network and return the result 1 if the optimal player wins 0 if a draw and -1 if the neural network wins (this never happens, since if all players played optimally there can only ever be a draw).

If you play every level of the NN player against the optimal player many times, you can plot what percentage of games a given level of player can draw or win, the plot hence shows the effectiveness of learning.

```julia
addprocs(4) # add 4 more Julia processors
using Distributed

stats = pmap(1:50) do i # run the evaluation in parallel
        gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
        nn = load_model("models_48_81boot", string(i), gametype)

        plays = [play_optimal(gametype, nn) for i=1:20]
    end

end


using Gadfly
Gadfly.plot(y=(20 .- sum.(stats)) / 20 * 100, x=[1:50;] .* 10,
    Geom.point, Geom.smooth, Guide.xlabel("Games trained on "), Guide.ylabel("NN vs. Optimal (% not lost)"))
```

<img src="https://user-images.githubusercontent.com/25916/47750325-8f5f5f80-dc65-11e8-9a23-6dd0475d238a.png" width="600">

## Appendix

We initially trained the network on 10000 games without considering the option `start_training_after` which is by default set to 50000 -- these are the number of sample branches explored. So our training did not start to learn anything until it played about 6000 games, but was sampling games to assign priors for the monte carlo tree search. However, once the training started the priors accumulated from the 6000 games were enough to quickly learn the optimal strategy. The plot below shows this.


<img src="https://user-images.githubusercontent.com/25916/47750905-cc782180-dc66-11e8-8ac8-b38a4154f26b.png" width="600">

However for the purpose of the examples here we went with the much lesser trained network of a setup where we only use 81 initial games to set the prior for the monte-carlo sampling. Also we only trained on 500 games and captured a snaptshot of the agent more frequently. This makes for a much more human-accessible example. You can actually play against the agent and see how dumb it started out and how it learned specific moves.


### Don Knuth takes a crack at TicTacToe in 10000 bytes

Here is Donald Knuth talking about solving the game of tic tac toe by training against an optimal player https://www.youtube.com/watch?v=_c3dKYrjj2Q . The idea of learning from self-play in AlphaGo are certainly not new, but the cleverness is in the monte carlo tree search. And of course the enormous amount of compute power required to train the network.

## References

- **Mastering the game of Go without human knowledge**  -- Nature (Oct. 2017)
David Silver, Julian Schrittwieser, Karen Simonyan, Ioannis Antonoglou, Aja Huang, Arthur Guez, Thomas Hubert, Lucas Baker, Matthew Lai, Adrian Bolton, Yutian Chen, Timothy Lillicrap, Fan Hui, Laurent Sifre, George van den Driessche, Thore Graepel & Demis Hassabis

- **AlphaGo Zero Explained - On AI** blog post https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/

- **AlphaGo.jl** - Julia implementation of the AlphaGo Zero problem on Go and Gomoku -- a generalization of tic-tac-toe